Ad-hoc computations with Futures
------------------------------------

While many parallel applications can be described as maps, some can be more complex.
In this section we look at the asynchronous Future interface,
which provides a simple API for ad-hoc parallelism.
This is useful for when your computations don't fit a regular pattern.

### Objectives

*  Use the `concurrent.futures` function `submit` to perform ad-hoc parallel computing

### Requirements

*  Pandas
*  concurrent.futures (standard in Python 3, `pip install futures` in Python 2)


    pip install snakeviz
    pip install futures

### Application

Given our HDF5 files from the last section we want to find the two datasets with the greatest pair-wise correlation.  This forces us to consider all $n\times(n-1)$ possibilities.

As before we start with a sequential solution.

In [ ]:
from glob import glob
import pandas as pd
import os

filenames = sorted(glob(os.path.join('..', 'data', 'json', '*.h5')))  # ../data/json/*.json
filenames[:5]

In [ ]:
series = {}
for fn in filenames:
    series[fn] = pd.read_hdf(fn)['close']

In [ ]:
%%time

results = {}

for a in filenames:
    for b in filenames:
        if a != b:
            results[a, b] = series[a].corr(series[b])
            
((a, b), corr) = max(results.items(), key=lambda kv: kv[1])
print("%s matches with %s with correlation %f" % (a, b, corr))

### Visually inspect correlated series

We use matplotlib to visually inspect the highly correlated timeseries

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 4))
plt.plot(series[a] / series[a].max())
plt.plot(series[b] / series[b].max())
plt.xticks(visible=False);

### Analysis

This computation starts out with an embarrassingly parallel part loading data from disk:

```python
series = {}
for fn in filenames:
    series[fn] = pd.read_hdf(fn)['x']
```

It follows with a doubly nested for loop with an if statement.  

```python
results = {}

for a in filenames:
    for b in filenames:
        if a != b:
            results[a, b] = series[a].corr(series[b])
```

It *is* possible to solve this problem with `map`, but it requires some cleverness.  Instead we'll learn `submit`, an interface to start individual function calls asynchronously.

It finishes with a reduction on small data.  We're not going to care about parallelizing this (it's already very fast.)

```python
((a, b), corr) = max(results.items(), key=lambda kv: kv[1])
```

### Executor.submit

The `submit` method starts a computation in a separate thread or process and immediately gives us a `Future` object that refers to the result.  At first, the future is pending.  Once the function completes the future is finished. 

We collect the result of the task with the `.result()` method,
which does not return until the results are available.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
e = ThreadPoolExecutor(4)

from time import sleep

def slowadd(a, b, delay=1):
    sleep(delay)
    return a + b

future = e.submit(slowadd, 1, 2)
future

In [ ]:
future.result()

### Submit many tasks, receive many futures

Because submit returns immediately we can submit many tasks all at once and they will execute in parallel.

In [ ]:
%%time
results = [slowadd(i, i, delay=1) for i in range(10)]

In [ ]:
%%time
futures = [e.submit(slowadd, 1, 1, delay=1) for i in range(10)]
results = [f.result() for f in futures]

### Submit different tasks

The virtue of submit is that you can submit different functions and you can perform a bit of logic on each input.

### Exercise: parallelize the following code with e.submit

1.  Replace the `results` list with a list called `futures`
2.  Replace calls to `slowadd` and `slowinc` with `e.submit` calls on those functions
3.  At the end, block on the computation by recreating the `results` list by calling `.result()` on each future in the `futures` list.

In [ ]:
%%time

### Sequential Version

def slowadd(a, b, delay=1):
    sleep(delay)
    return a + b

def slowsub(a, b, delay=1):
    sleep(delay)
    return a - b

results = []
for i in range(5):
    for j in range(5):
        if i < j:
            results.append(slowadd(i, j, delay=1))
        elif i > j:
            results.append(slowsub(i, j, delay=1))

In [ ]:
%%time

### Parallel Version

# TODO

In [ ]:
%load solutions/submit-1.py

### Conclusion on submit

*  Submit fires off a single function call in the background, returning a future.  
*  When we combine submit with a single for loop we recover the functionality of map.  
*  When we want to collect our results we replace each of our futures, `f`, with a call to `f.result()`
*  We can combine submit with multiple for loops and other general programming to get something more general than map.


### Exercise: Parallelize pair-wise correlations with `e.submit`


In [ ]:
%%time

### Sequential Code

results = {}

for a in filenames:
    for b in filenames:
        if a != b:
            results[a, b] = series[a].corr(series[b])

In [ ]:
%%time

### Parallel Code

futures = ... # TODO

# TODO

results = ... # TODO

In [ ]:
%load solutions/submit-2.py

### Exercise: Threads vs Processes

Try the exercise above using Processes vs Threads by replacing `e` with a ProcessPoolExecutor:

#### Before

```python
from concurrent.futures import ThreadPoolExecutor
e = ThreadPoolExecutor(4)
```

#### After

```python
from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor(4)
```

How does performance vary?  We'll talk more about the tradeoffs between threads and processes later on in the tutorial.

### Exercise: Break Python by loading the data in parallel

The HDF5 library we use to load our data is not threadsafe and can cause our entire Python session to crash.

In [ ]:
%%time
from concurrent.futures import ThreadPoolExecutor
e = ThreadPoolExecutor(4)

dfs = e.map(pd.read_hdf, filenames)
series = e.map(lambda df: df['close'], dfs)
series = dict(zip(filenames, series))

Conclusion
-----------

*  We learned how `e.submit` can help us to parallelize more complex applications
*  We used `e.submit` to compute pairwise collelations in parallel
*  We learned that this didn't actually speed up our code very much
*  We compared threads against processes to see some performance differences
*  We crashed our Python session by using threads with unsafe code, warning us that parallelism is sometimes dangerous